In [1]:
#!jupyter nbconvert --to=python load_interp_Chauetal.ipynb

In [ ]:
class CHAUdata:
    '''
    Load SOCAT file and interpolate to FESOM mesh.
    
    Careful: Output still needs to be masked with an ocean bathymetry mask, e.g. with FESOM output!
    
    In: ncfile,ncvariable,meshpath
    
    Out: class SOCATdata with self.glodap_int containing interpolated SOCAT ncvariable
    '''
    
    def __init__(self,runname,resultpath,mesh,ncfile,ncvariable,first_year,last_year,get_overview=False):
    
        self.runid = runname
        self.resultpath = resultpath
        self.mesh = mesh
        self.ncfile = ncfile
        self.ncvariable = ncvariable
        self.get_overview = get_overview
        self.fyear = first_year
        self.lyear = last_year

        import matplotlib.pyplot as plt
        import numpy as np
        import netCDF4
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        from numba import njit
        #import skill_metrics as sm
        #import cartopy.crs as ccrs
        #import pickle
        import pyfesom2 as pf
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)
        
        # load NetCDF ------------------------------------------------------------------------------------
        print('***\nLoading Chau et al. file: {0}\n***'.format(self.ncfile))
        
        f = Dataset(ncfile, 'r')
        if self.ncvariable == 'fgco2':
            VARraw  = f.variables['fgco2'][:,:,:]      # fCO2 observations
        elif self.ncvariable == 'spco2':
            VARraw  = f.variables['spco2'][:,:,:]
        VARraw  = np.ma.filled(VARraw, np.nan)  # From masked array to numpy array
        VARraw = np.transpose(VARraw, (1, 2, 0)) # now ordered lat, lon, time

        if self.ncvariable == 'fgco2':
            VARraw  = - VARraw * 3600 * 24 * 365 * 1000 / 12.01 # converting kg/m2/s to molC/m2/year
        elif self.ncvariable == 'spco2':
            VARraw *= 9.8692326671 # conversion from Pascal to uatm
        lon_socat  = f.variables['longitude'][:]           
        lon_socat[lon_socat>180.] = lon_socat[lon_socat>180.]-360. # -180 to 180, okay.
        # print min(lon_socat), max(lon_socat)
        lat_socat  = f.variables['latitude'][:]
        hours_socat = f.variables['time'][:]          # hours since 1950-01-01 00:00:00
        time_convert = netCDF4.num2date(hours_socat[:], "hours since 1950-01-01 00:00:00", "Gregorian")

        year_socat = hours_socat/(365*24)+1950              # fractional years
        
        if (self.fyear < 1985):
            print("\n***\n no Chau et al. data to be plotted before 1985\n***")  
            self.fyear = 1985
        if (self.lyear > 2020):
            print("\n***\n no Chau et al. data to be plotted after 2020\n***")  
            self.lyear = 2020
        if ((self.lyear < 1985) or (self.fyear>2020)):
            print("\n***\n Chau et al. data only available for the 1985-2020 period\n*** no matchup possible with FESOM period {0}-{1}\n***\n*** comparing to SOCAT climatology\n***\n***\n***".format(self.fyear,self.lyear))  
            self.lyear = 2020
            self.fyear = 1985
            
        # Define regular grid
        X360, Y180 = np.meshgrid(lon_socat,lat_socat) 
        
        # Select time period
        VARsoc = VARraw[:,:,(year_socat>=self.fyear)&(year_socat<(self.lyear+1))]
        time_convert_select = time_convert[(year_socat>=self.fyear)&(year_socat<(self.lyear+1))]
        
        print("\n***\n years extracted in Chau et al. {0}-{1} \n***".format(self.fyear,self.lyear))
        
        # Loop over months
        #@njit
        def derive_matchups(VARsoc,time_convert_select,mesh,X360,Y180):
            socat_int = np.empty((len(mesh.x2),len(time_convert_select)))
            for i in range(len(time_convert_select)):
                ind   = ~np.isnan(VARsoc[:,:,i])
                aux = VARsoc[:,:,i]
                data1 = griddata((X360.ravel(), Y180.ravel()), aux.ravel(), (mesh.x2,mesh.y2), method='nearest')
                data1 = np.ma.filled(data1, np.nan)
                socat_int[:,i] = data1
            return socat_int
        
        socat_int = derive_matchups(VARsoc,time_convert_select,mesh,X360,Y180)
        
        # calculate mean over that time-period:
        VARsoc_ave = np.nanmean(VARsoc,axis=2)#,dtype=ndarray)
        ind   = ~np.isnan(VARsoc_ave)

        # Interpolation to fesom's grid
        aux = VARsoc_ave
        
        #data1 = griddata((X360[ind], Y180[ind]), aux[ind], (mesh.x2,mesh.y2), method='nearest')
        data1 = griddata((X360.ravel(), Y180.ravel()), aux.ravel(), (mesh.x2,mesh.y2), method='nearest')
        socat_int_mean = np.ma.filled(data1, np.nan)
                
        #socat_int = np.swapaxes(socat_int,0,1) # adjust axes layout to FESOM output
        self.socat_int_mean = socat_int_mean
        self.socat_int = socat_int
        self.time = time_convert_select